## Grab SafeGraph Social Distancing/Mobility data from AWS 

In [5]:
from bs4 import BeautifulSoup as bs
from bs4 import Tag, NavigableString
import re
import requests as r
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os
import sys
from state_cleaner import *
from selenium import webdriver
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'C:\\Users\\Noah\\CovidMobile'

In [10]:
#Configure the aws profile **check the slack for new access keys weekly!
#! aws configure --profile safegraph

/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/notebooks


In [ ]:
#Downloads all current available data social distancing data: *takes a while to run
#! aws s3 sync s3://sg-c19-response/social-distancing/v2/ ./SafeGraph_data --profile safegraph

In [5]:
full_df = pd.read_csv('./data/compiled_2020-05-15.csv')

In [6]:
print(full_df['date'].min(), full_df['date'].max())

2020-02-15 2020-05-07


In [10]:
#Create list of dates to loop through
dates_obj = list(pd.date_range(start=full_df['date'].min(),end=full_df['date'].max()))
dates_slashes = [x.strftime("%Y/%m/%d") for x in dates_obj]
dates_dashes = [x.strftime("%Y-%m-%d") for x in dates_obj]

#d_s = dates_slashes[round(len(dates_slashes)/2):]
#d_d = dates_dashes[round(len(dates_dashes)/2):]

In [11]:
sdm_df = pd.DataFrame()

In [12]:
#Read in all of the csvs for each date:
for date1, date2 in zip(dates_slashes, dates_dashes):
    df = pd.read_csv("SafeGraph_data/{}/{}-social-distancing.csv.gz".format(date1,date2))
    df = df[["origin_census_block_group", "date_range_start", "date_range_end", 
                                 "device_count", "completely_home_device_count", "part_time_work_behavior_devices", 
                                 "full_time_work_behavior_devices"]]
    
    df = df.astype({'origin_census_block_group':'str'})
    df['FIPS'] = df.origin_census_block_group.str.slice(start=0, stop=5) # county is the first 5 digits of the CBG
    df['date'] = df.date_range_start.str.slice(start=0, stop=10)
    
    sdm_columns = ['device_count', 'completely_home_device_count','part_time_work_behavior_devices', 'full_time_work_behavior_devices']
    df_by_county = df.groupby(['FIPS', 'date'])[sdm_columns].sum().reset_index()
    
    sdm_df = sdm_df.append(df_by_county)
    print(date1)

2020/02/15
2020/02/16
2020/02/17
2020/02/18
2020/02/19
2020/02/20
2020/02/21
2020/02/22
2020/02/23
2020/02/24
2020/02/25
2020/02/26
2020/02/27
2020/02/28
2020/02/29
2020/03/01
2020/03/02
2020/03/03
2020/03/04
2020/03/05
2020/03/06
2020/03/07
2020/03/08
2020/03/09
2020/03/10
2020/03/11
2020/03/12
2020/03/13
2020/03/14
2020/03/15
2020/03/16
2020/03/17
2020/03/18
2020/03/19
2020/03/20
2020/03/21
2020/03/22
2020/03/23
2020/03/24
2020/03/25
2020/03/26
2020/03/27
2020/03/28
2020/03/29
2020/03/30
2020/03/31
2020/04/01
2020/04/02
2020/04/03
2020/04/04
2020/04/05
2020/04/06
2020/04/07
2020/04/08
2020/04/09
2020/04/10
2020/04/11
2020/04/12
2020/04/13
2020/04/14
2020/04/15
2020/04/16
2020/04/17
2020/04/18
2020/04/19
2020/04/20
2020/04/21
2020/04/22
2020/04/23
2020/04/24
2020/04/25
2020/04/26
2020/04/27
2020/04/28
2020/04/29
2020/04/30
2020/05/01
2020/05/02
2020/05/03
2020/05/04
2020/05/05
2020/05/06
2020/05/07


In [13]:
# convert numerical columns
sdm_df= sdm_df.astype({'device_count':'int','completely_home_device_count':'int'})

In [14]:
# compute new metrics
sdm_df['leaving_home'] = sdm_df['device_count'] - sdm_df['completely_home_device_count']
sdm_df['pct_leaving_home'] = sdm_df['leaving_home'] / sdm_df['device_count'] 

sdm_df['pct_social_distancing'] = sdm_df['completely_home_device_count']/(sdm_df['device_count']-sdm_df['full_time_work_behavior_devices']-sdm_df['part_time_work_behavior_devices'])
  
sdm_df.head()

,FIPS,date,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,leaving_home,pct_leaving_home,pct_social_distancing
0,10001,2020-02-15,12286,3445,676,196,8841,0.719600,0.301822
1,10003,2020-02-15,31356,8501,1695,658,22855,0.728888,0.293108
2,10005,2020-02-15,14123,3694,834,278,10429,0.738441,0.283914
3,10010,2020-02-15,5708,1152,622,180,4556,0.798178,0.234815
4,10030,2020-02-15,23838,5358,2407,805,18480,0.775233,0.259769


In [15]:
# Merge with the full data set:
sdm_df = sdm_df.sort_values(by='FIPS')
sdm_df = sdm_df.astype({'FIPS':'float64'})
combined_df = full_df.merge(sdm_df,on = ['FIPS','date'])
combined_df

,country_region_code,country_region,state_x,county_x,date,retail,food_drugs,park,transit,work,...,county_y,first_case_date,_merge,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,leaving_home,pct_leaving_home,pct_social_distancing
0,US,United States,Delaware,Kent County,2020-02-15,5.0,4.0,3.0,NaN,0.0,...,NaN,NaN,left_only,12286,3445,676,196,8841,0.719600,0.301822
1,US,United States,Delaware,Kent County,2020-02-16,5.0,2.0,20.0,NaN,-1.0,...,NaN,NaN,left_only,11870,3605,715,229,8265,0.696293,0.329947
2,US,United States,Delaware,Kent County,2020-02-17,10.0,2.0,NaN,-3.0,-22.0,...,NaN,NaN,left_only,11290,3017,969,567,8273,0.732772,0.309309
3,US,United States,Delaware,Kent County,2020-02-18,7.0,-1.0,NaN,14.0,4.0,...,NaN,NaN,left_only,10609,2304,1428,1073,8305,0.782826,0.284164
4,US,United States,Delaware,Kent County,2020-02-19,4.0,0.0,-6.0,3.0,1.0,...,NaN,NaN,left_only,11413,2772,1393,1048,8641,0.757119,0.308961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197665,US,United States,Wyoming,Weston County,2020-05-01,NaN,NaN,NaN,NaN,-32.0,...,NaN,NaN,left_only,346,122,18,14,224,0.647399,0.388535
197666,US,United States,Wyoming,Weston County,2020-05-04,NaN,NaN,NaN,NaN,-34.0,...,NaN,NaN,left_only,329,137,27,8,192,0.583587,0.465986
197667,US,United States,Wyoming,Weston County,2020-05-05,NaN,NaN,NaN,NaN,-34.0,...,NaN,NaN,left_only,317,103,28,5,214,0.675079,0.362676
197668,US,United States,Wyoming,Weston County,2020-05-06,NaN,NaN,NaN,NaN,-35.0,...,NaN,NaN,left_only,345,129,27,12,216,0.626087,0.421569


In [16]:
sdm_df.to_csv('./SafeGraph_data/sg_mobility_updated_{}.csv'.format(str(datetime.now().date())))

In [17]:
full_df.to_csv('./SafeGraph_data/compiled+sg_{}.csv'.format(str(datetime.now().date())))

### Start here if you already have the sg_mobility_updated csv file

In [20]:
sdm_df = pd.read_csv('SafeGraph_data/sg_mobility_updated_2020-05-18.csv')
full_df = pd.read_csv('data/compiled_2020-05-15.csv')

In [21]:
full_df = full_df.dropna(subset = ['FIPS'])

In [23]:
#Merge:
sdm_df = sdm_df.sort_values(by='FIPS')
sdm_df = sdm_df.astype({'FIPS':'float64'})
combined_df = full_df.merge(sdm_df,on = ['FIPS','date'],how='left')
combined_df

,country_region_code,country_region,state_x,county_x,date,retail,food_drugs,park,transit,work,...,first_case_date,_merge,Unnamed: 0,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,leaving_home,pct_leaving_home,pct_social_distancing
0,US,United States,Alabama,AL,2020-02-15,5.0,2.0,39.0,7.0,2.0,...,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US,United States,Alabama,AL,2020-02-16,0.0,-2.0,-7.0,3.0,-1.0,...,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US,United States,Alabama,AL,2020-02-17,3.0,0.0,17.0,7.0,-17.0,...,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US,United States,Alabama,AL,2020-02-18,-4.0,-3.0,-11.0,-1.0,1.0,...,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US,United States,Alabama,AL,2020-02-19,4.0,1.0,6.0,4.0,1.0,...,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223916,US,United States,Wyoming,Weston County,2020-05-01,NaN,NaN,NaN,NaN,-32.0,...,NaN,left_only,3013.0,346.0,122.0,18.0,14.0,224.0,0.647399,0.388535
223917,US,United States,Wyoming,Weston County,2020-05-04,NaN,NaN,NaN,NaN,-34.0,...,NaN,left_only,3012.0,329.0,137.0,27.0,8.0,192.0,0.583587,0.465986
223918,US,United States,Wyoming,Weston County,2020-05-05,NaN,NaN,NaN,NaN,-34.0,...,NaN,left_only,3013.0,317.0,103.0,28.0,5.0,214.0,0.675079,0.362676
223919,US,United States,Wyoming,Weston County,2020-05-06,NaN,NaN,NaN,NaN,-35.0,...,NaN,left_only,3013.0,345.0,129.0,27.0,12.0,216.0,0.626087,0.421569


In [26]:
#get rid of weird random column
combined_df = combined_df.drop(columns = ['Unnamed: 0'])

combined_df.to_csv('data/compiled_w_sg_{}.csv'.format(str(datetime.now().date())))

KeyboardInterrupt: 

## Get SafeGraph Census data, using their Google Drive folder

In [4]:
from google.colab import auth
auth.authenticate_user()

FileNotFoundError: [Errno 2] No such file or directory: 'gcloud': 'gcloud'

In [2]:
# special libraries to allow file access
from google.colab import drive as mountGoogleDrive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

/opt/anaconda3/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
# These commands allow you to read directly from SafeGraph's public GoogleDrive containing Census Data and Sample Data
auth.authenticate_user()  # Authenticate and create the PyDrive client. 
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("You are fully authenticated and can edit and re-run everything in the notebook. Enjoy!")

FileNotFoundError: [Errno 2] No such file or directory: 'gcloud': 'gcloud'